In [1]:
import os
import uuid
import logging
from dotenv import load_dotenv

from dapr_agents.tool.mcp import MCPClient

logging.basicConfig(level=logging.INFO)
load_dotenv()

True

In [ ]:
# ---- MCP Jupyter server env (REMOTE) ----
NOTEBOOK_NAME = f"mcp_{uuid.uuid4().hex[:8]}.ipynb"

ENV = {
    **os.environ,
    "MCP_JUPYTER_SESSION_MODE": "server",
    "MCP_JUPYTER_BASE_URL": os.getenv("JUPYTER_BASE_URL"),
    "MCP_JUPYTER_TOKEN": os.getenv("JUPYTER_TOKEN"),
    "MCP_JUPYTER_KERNEL_NAME": os.getenv("JUPYTER_KERNEL_NAME", "python3"),
    "MCP_JUPYTER_NOTEBOOK_PATH": os.getenv("JUPYTER_NOTEBOOK_PATH", NOTEBOOK_NAME),
    "MCP_JUPYTER_TOOLSETS": "postgresql",
}

In [3]:
client = MCPClient()

await client.connect_stdio(
    server_name="local",
    command="uvx",
    #args=["mcp-jupyter-notebook", "--transport", "stdio"],
    args=[
        "--refresh",
        "--from",
        "../../mcp-jupyter-notebook",
        "mcp-jupyter-notebook",
        "--transport",
        "stdio",
    ],
    env=ENV,
)

INFO:dapr_agents.tool.mcp.client:Loaded 7 tools from server 'local'
INFO:dapr_agents.tool.mcp.client:Loaded 0 prompts from server 'local': 
INFO:dapr_agents.tool.mcp.client:Successfully connected to MCP server 'local' (ephemeral mode)


In [4]:
from dapr_agents.tool import AgentTool
from typing import List

tools: List[AgentTool] = client.get_all_tools()
print("🔧 Available tools:", [t.name for t in tools])

🔧 Available tools: ['LocalNotebook.Markdown.Add', 'LocalNotebook.Code.Run', 'LocalNotebook.Packages.Add', 'LocalDb.Postgresql.Query.ToDf', 'LocalDb.Postgresql.Schema.ListTables', 'LocalDb.Postgresql.Schema.ListColumns', 'LocalDb.Postgresql.Schema.Tree']


In [5]:
tools[5].to_function_call()

{'type': 'function',
 'function': {'name': 'LocalDb.Postgresql.Schema.ListColumns',
  'description': 'Hidden: list columns for (schema, table). (from MCP server: local)',
  'parameters': {'properties': {'schema_name': {'description': 'The schema name containing the table',
     'type': 'string'},
    'table': {'description': 'The table name to list columns for',
     'type': 'string'}},
   'required': ['schema_name', 'table'],
   'type': 'object'}}}